In [1]:
import numpy as np

import os, math, pdb, glob, os, sys, natsort, h5py, mne
#Choose GPU 0 as a default
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Activation
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.models import Model
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

import sys
sys.path.append('/home/zsteineh/cnn_hilbert/cnn_hilbert_workspace')
import hilbert_DL_utils
from hilbert_DL_utils import load_data
sys.path.append('/home/zsteineh/ez-ssl/pretasks')
from make_ecog_data import get_speech_epochs 
import pickle

In [10]:
fold = 2
datatype = "speech" #model
pretask_type = 'sig_tran' #sig_tran
model_dir = '/data1/users/gsquist/state_decoder/accuracy_outputs/a0f66459/class_ssl/'
model_name = pretask_type+'_'+datatype+'_htnet_fold_'+str(fold)+'.h5'

model_fname = model_dir + model_name

norm_rate = 0.25
wrist_lp = '/data1/users/stepeter/cnn_hilbert/ecog_data/xarray/'
speech_lp = '/data2/users/gsquist/speech_project/epochd_ecog_data/'
pats_ids_in = ['a0f66459']
test_day = 'last'
n_chans_all=64
tlim=[-1,1]
n_folds = 1

optimizer='adam'
loss='binary_crossentropy'
patience = 15
early_stop_monitor='val_loss'
epochs=64
sp = '/home/zsteineh/ez_ssl_results/'
chckpt_path = sp+'checkpoint_gen_tl_'+datatype+'_'+pats_ids_in[0]+'_fold'+str(fold)+'.h5'

In [11]:
if datatype == "wrist":
        X,y,x_test,y_test,sbj_order_all,sbj_order_test_last = load_data(pats_ids_in, wrist_lp,
                                                              n_chans_all=n_chans_all,
                                                              test_day=test_day, tlim=tlim)
elif datatype == "speech":
    sbj = pats_ids_in[0]
    sigs, labs = [], []
    f_load = natsort.natsorted(glob.glob(speech_lp+sbj+'/speak_silent/'+'*_epo.fif'))
    print("for sjb",sbj, "f_load is",f_load)
    eps_to_file = {}
    for i, f in enumerate(f_load):
        eps = f_load[i].split('/')[-1].split('_')[3][:3]
        eps_to_file[eps] = f_load[i]
    max_eps, min_eps = max(eps_to_file, key=eps_to_file.get), min(eps_to_file, key=eps_to_file.get)
    epochs_train = mne.read_epochs(eps_to_file[max_eps])
    epochs_test = mne.read_epochs(eps_to_file[min_eps])
    X = epochs_train.get_data()
    x_test = epochs_test.get_data()

    y = epochs_train.events[:,-1]
    y_test = epochs_test.events[:,-1]

#     Y_trainval_cat = np_utils.to_categorical(labels_train)
#     Y_test = np_utils.to_categorical(labels_test)

for sjb a0f66459 f_load is ['/data2/users/gsquist/speech_project/epochd_ecog_data/a0f66459/speak_silent/a0f66459_speak_silent_163events_5_epo.fif', '/data2/users/gsquist/speech_project/epochd_ecog_data/a0f66459/speak_silent/a0f66459_speak_silent_243events_4_epo.fif']


<ipython-input-11-e396d24e6fc7>:15: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epochs_train = mne.read_epochs(eps_to_file[max_eps])
<ipython-input-11-e396d24e6fc7>:16: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epochs_test = mne.read_epochs(eps_to_file[min_eps])


In [12]:
nb_classes = len(np.unique(y))
order_inds = np.arange(len(y))
np.random.shuffle(order_inds)
X = X[order_inds,...]
y = y[order_inds]
order_inds_test = np.arange(len(y_test))
np.random.shuffle(order_inds_test)
x_test = x_test[order_inds_test,...]
y_test = y_test[order_inds_test]
y2 = np_utils.to_categorical(y-1)
y_test2 = np_utils.to_categorical(y_test-1)
X2 = np.expand_dims(X,1)
X_test2 = np.expand_dims(x_test,1)

split_len = int(X2.shape[0]*0.2)
last_epochs = np.zeros([n_folds,2])

val_inds = np.arange(0,split_len)+(0*split_len)
train_inds = np.setdiff1d(np.arange(X2.shape[0]),val_inds) #take all events not in val set

x_train = X2[train_inds,...]
y_train = y2[train_inds]
x_val = X2[val_inds,...]
y_val = y2[val_inds]

In [13]:
pretask_model = tf.keras.models.load_model(model_fname)
pretask_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 64, 3001)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 8, 64, 3001)       512       
_________________________________________________________________
lambda (Lambda)              (None, 8, 64, 3001)       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 8, 64, 3001)       32        
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 16, 1, 3001)       1024      
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 1, 3001)       64        
_________________________________________________________________
activation (Activation)      (None, 16, 1, 3001)       0     

In [ ]:
if pretask_type == 'rel_pos':
    sig_tran_model_fname = model_dir + 'sig_tran_' + datatype + '_htnet_fold_'+str(fold)+'.h5'
    sig_tran_pretask_model = tf.keras.models.load_model(sig_tran_model_fname)
    x = sig_tran_pretask_model.layers[-4].output
    x = Flatten(name = 'flatten2')(x)
    x = Dense(nb_classes, name = 'dense2', kernel_constraint = max_norm(norm_rate))(x)
    softmax = Activation('softmax', name = 'softmax2')(x)

    transfer_model = Model(inputs=sig_tran_pretask_model.input, outputs=softmax)
    transfer_model.load_weights(model_fname, by_name=True)

else:
    x = pretask_model.layers[-3].output
    # x = Flatten(name = 'flatten2')(x)
    x = Dense(nb_classes, name = 'dense', kernel_constraint = max_norm(norm_rate))(x)
    softmax = Activation('softmax', name = 'softmax')(x)

    transfer_model = Model(inputs=pretask_model.input, outputs=softmax)

# Set only last 3 layers to be trainable
for l in transfer_model.layers:
    l.trainable = False
for l in transfer_model.layers[-3:]:
    l.trainable = True #train last 3 layers
    

transfer_model.get_layer('depthwise_conv2d').trainable = True

transfer_model.summary()

In [ ]:
transfer_model.compile(loss=loss, optimizer=optimizer, metrics = ['accuracy'])
checkpointer = ModelCheckpoint(filepath=chckpt_path,verbose=1,save_best_only=True)
early_stop = EarlyStopping(monitor=early_stop_monitor, mode='min',
                               patience=patience, verbose=0)

In [ ]:
# run tests before finetuning model
acc_lst = []
preds = transfer_model.predict(x_train).argmax(axis = -1) 
acc_lst.append(np.mean(preds == y_train.argmax(axis=-1)))
preds = transfer_model.predict(x_val).argmax(axis=-1)
acc_lst.append(np.mean(preds == y_val.argmax(axis=-1)))
preds = transfer_model.predict(X_test2).argmax(axis = -1)
acc_lst.append(np.mean(preds == y_test2.argmax(axis=-1)))

print(np.asarray(acc_lst))

In [ ]:
h = transfer_model.fit(x_train, y_train, batch_size = 16, epochs = epochs, 
                        verbose = 2, validation_data=(x_val, y_val),
                        callbacks=[checkpointer,early_stop])

In [ ]:
transfer_model.load_weights(chckpt_path)
acc_lst = []
preds = transfer_model.predict(x_train).argmax(axis = -1) 
acc_lst.append(np.mean(preds == y_train.argmax(axis=-1)))
preds = transfer_model.predict(x_val).argmax(axis=-1)
acc_lst.append(np.mean(preds == y_val.argmax(axis=-1)))
preds = transfer_model.predict(X_test2).argmax(axis = -1)
acc_lst.append(np.mean(preds == y_test2.argmax(axis=-1)))

print(np.asarray(acc_lst))

In [ ]:
model_cloned = tf.keras.models.clone_model(transfer_model)
acc_lst = []
preds = model_cloned.predict(x_train).argmax(axis = -1) 
acc_lst.append(np.mean(preds == y_train.argmax(axis=-1)))
preds = model_cloned.predict(x_val).argmax(axis=-1)
acc_lst.append(np.mean(preds == y_val.argmax(axis=-1)))
preds = model_cloned.predict(X_test2).argmax(axis = -1)
acc_lst.append(np.mean(preds == y_test2.argmax(axis=-1)))
print(acc_lst)

In [ ]:
chckpt_path = sp+'checkpoint_gen_tsupervised_'+pats_ids_in[0]+'_fold'+str(fold)+'.h5'
model_cloned.compile(loss=loss, optimizer=optimizer, metrics = ['accuracy'])
checkpointer = ModelCheckpoint(filepath=chckpt_path,verbose=1,save_best_only=True)
early_stop = EarlyStopping(monitor=early_stop_monitor, mode='min',
                               patience=patience, verbose=0)
h = model_cloned.fit(x_train, y_train, batch_size = 16, epochs = epochs, 
                        verbose = 2, validation_data=(x_val, y_val),
                        callbacks=[checkpointer,early_stop])

In [ ]:
model_cloned.load_weights(chckpt_path)
acc_lst = []
preds = model_cloned.predict(x_train).argmax(axis = -1) 
acc_lst.append(np.mean(preds == y_train.argmax(axis=-1)))
preds = model_cloned.predict(x_val).argmax(axis=-1)
acc_lst.append(np.mean(preds == y_val.argmax(axis=-1)))
preds = model_cloned.predict(X_test2).argmax(axis = -1)
acc_lst.append(np.mean(preds == y_test2.argmax(axis=-1)))
print(acc_lst)